In [ ]:
# preprocessing/image_preprocessing.ipynb

In [2]:
!pip install wget

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9680 sha256=741213e99b630f601911c5e02db1fe19ee25088477d212611cfa09032d6eae74
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\40\b3\0f\a40dbd1c6861731779f62cc4babcb234387e11d697df70ee97
Successfully built wget


In [3]:
url = "https://download.pytorch.org/tutorial/hymenoptera_data.zip"

In [4]:
import os
import wget
# 디렉토리 생성
os.makedirs("data", exist_ok=True)
# url의 파일을 다운로드
zip_path = wget.download(url, out="data/hymenoptera_data.zip")
# 파라미터 (다운받을 URL,  out="받은파일 저장경로")
# 다운받은 파일의 경로를 반환.
zip_path

100% [........................................................................] 47286322 / 47286322

'data/hymenoptera_data.zip'

In [6]:
##### 다운받은 zip파일 압축풀기
# zip 압축 다루는 표준모듈
from zipfile import ZipFile
# 압축파일연결 -> 압축풀기 -> 연결닫기 
with ZipFile(zip_path) as zf:     # 풀 압축파일의 경로를 넣어서 객체 생성
    zf.extractall("data")  # "data": 디렉토리 아래 풀기. 생략시 현재 작업경로 아래 푼다.

In [15]:
## glob.glob() # 이미지의 경로를 반환. 
#                               => wild card 문자를 이용해서 다수 파일의 경로를 한번에 조회가능
#           파일: * (모든 문자들),  디렉토리: ** (모든 하위 디렉토리들)
#           *.jpg : 확장자가 jpg인 모든 파일, 
#            a*.exe : a로 시작하는 이름의 모든 exe파일들.
#           c:\test\**   :  c:\test 아래있는 모든 하위 및 자손 디렉토리.(재귀적 조회)

#### data 아래 있는 모든 jpg 파일들의 경로
from glob import glob
# glob("data/**/*.jpg", recursive=True)
#### data 아래있는 디렉토리 
glob("data/**/") #, recursive=True)

['data\\hymenoptera_data\\']

In [ ]:
####### 복사 -> shutil 표준모듈.  
##  copy(src, target)   - 파일 + 권한정보
##  copy2(src, target) - 파일 + 메타정보(생성일자, 수정일자..)
##  copyfile(src, target) - 파일
##  copytree(src, target)  - 디렉토리 복사.

In [19]:
####### data 및 그 하위 경로 아래 모든 이미지파일들(.jpg)을 data 폴더 아래로 카피.
# 이미지 파일들이 있는 디렉토리 조회
import shutil

img_dir_path = glob("data/**/")[0]  # 카피후 디렉토리 삭제를 위해서 경로 저장.
file_list = glob("data/**/*.jpg", recursive=True)

# 복사
for file in file_list:
    shutil.copy2(file, "data")  # file --복사--> data

In [21]:
# 원본 이미지 파일들 디렉토리, 다운받은 압축파일 삭제
try:
    os.remove(zip_path) # 압축파일 삭제
except:
    print('삭제할 파일이 없음.')

try:
    shutil.rmtree(img_dir_path)
except:
    print("삭제할 디렉토리 없음")

삭제할 파일이 없음.
삭제할 디렉토리 없음


# 이미지 전처리 
- 이진화
- 좌우 이동
- 밝기 변경
- 엣지 디텍션
- 모든 이미지에 위 4가지 처리를 한 파일을 각각 생성.

In [26]:
##### 파일 경로의 정보 조회
fname = 'c:\\images\\data\\1030023514_aad5c608f9.jpg'
# 파일이름
print(os.path.basename(fname))
print(os.path.dirname(fname))
print(os.path.splitext("1030023514_aad5c608f9.jpg")) #파일명과  확장자를 분리

1030023514_aad5c608f9.jpg
c:\images\data
('1030023514_aad5c608f9', '.jpg')


In [30]:
import cv2 
import numpy as np

img_list = glob('data/*.jpg')
cnt = 0
for img_path in img_list:
    fname = os.path.basename(img_path) # 파일명 조회
    fname, ext = os.path.splitext(fname)   # 파일이름, 확장자를 분리해서 변수에 저장.
    # 읽기
    img = cv2.imread(img_path)
    ## 이진화
    ### grayscale 변환. 
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ### 이진화
    _, img_bin = cv2.threshold(img_gray, 127, 255, cv2.THRESH_BINARY)
    ### 저장
    save_path1 = f"data/{fname}_binary_{cnt:03d}{ext}"  # 03d: 0으로 세자리 맞춤. 1->001
    cv2.imwrite(save_path1,  img_bin)
    
    ## 좌우 이동 (20px 이동)
    affine_matrix = np.array([[1, 0, 20], [0, 1, 0]], dtype="float32")
    img_move = cv2.warpAffine(img, affine_matrix, (0, 0))
    save_path2 = f"data/{fname}_move_{cnt:03d}{ext}"
    cv2.imwrite(save_path2, img_move)

    ## 밝기 변경 (30 만큼 밝게)
    img_bright = cv2.add(img, 30)
    save_path3 = f"data/{fname}_bright_{cnt:03d}{ext}"
    cv2.imwrite(save_path3, img_bright)

    ## 엣지 검출
    img_edge = cv2.Canny(img_gray, 50, 150)
    save_path4 = f"data/{fname}_edge_{cnt:03d}{ext}"
    cv2.imwrite(save_path4, img_edge)

    cnt += 1
    